## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark


In [0]:
# import necessary libraries
import findspark
findspark.init()

In [0]:
# import pyspark
# from pyspark import SparkContext
# sc = pyspark.SparkContext('local[*]')

In [0]:
# initialize Spark Session
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.master("local[*]").getOrCreate()

## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="https://github.com/jammy-bot/dsc-resilient-distributed-datasets-rdd-lab-online-ds-ft-120919/blob/master/images/rdd_diagram.png?raw=1" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [4]:
data = list(range(1, 1001))
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [0]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [6]:
rdd = sc.parallelize(data, numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [7]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [8]:
# count
rdd.count()

1000

In [9]:
# first
rdd.first()

1

In [10]:
# take
rdd.take(10)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [11]:
# top
rdd.top(10)

[1000, 999, 998, 997, 996, 995, 994, 993, 992, 991]

In [12]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [13]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([6.39650841e-01, 1.81353728e+00, 3.78179794e-01, 8.84718660e-01,
       2.23945373e+00, 2.70253968e-01, 6.87641705e+00, 5.09316699e+00,
       4.24696497e+00, 4.52894076e+00, 1.98363163e-01, 9.93703585e+00,
       1.19374297e+01, 1.15418016e+01, 8.82599832e+00, 5.84384179e+00,
       1.15098391e+01, 1.25627222e+01, 1.05218632e+01, 8.21929205e+00,
       1.94805947e+01, 5.48705136e+00, 1.93127317e+01, 4.95925113e+00,
       2.27697764e+01, 1.86477278e+01, 1.53659816e+01, 2.51492117e+01,
       1.14713626e+01, 2.14913093e+01, 2.26105068e+01, 1.10394086e+01,
       2.65671701e+01, 2.43128567e+01, 2.03175254e+01, 1.11019808e+01,
       7.08277184e+00, 3.04157426e+01, 3.79863898e+01, 3.59979935e+01,
       3.80834721e+01, 1.28478014e+01, 3.07460071e+01, 2.82163361e+01,
       2.18171740e+01, 2.55897851e+01, 6.44746020e-01, 1.44491585e+01,
       2.67896027e+01, 4.91549714e+01, 1.38985736e+01, 3.24463249e+00,
       1.19581177e+01, 1.93826670e+01, 3.78924348e+01, 2.46064718e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [14]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.take(4)

[0.6396508406898006,
 1.8135372791296511,
 0.3781797936312401,
 0.8847186600850305]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [0]:
def sales_tax(num):
    return num*0.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [16]:
# perform action to retrieve rdd values
revenue_minus_tax.take(10)

[0.5884787734346166,
 1.668454296799279,
 0.3479254101407409,
 0.8139411672782281,
 2.060297430802508,
 0.24863365015159442,
 6.32630368530064,
 4.685713634537652,
 3.907207770853302,
 4.166625496238749]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [0]:
discounted = revenue_minus_tax.map(lambda x: x*0.9)

In [18]:
discounted.take(10)

[0.529630896091155,
 1.5016088671193513,
 0.3131328691266668,
 0.7325470505504053,
 1.8542676877222575,
 0.223770285136435,
 5.6936733167705755,
 4.217142271083888,
 3.516486993767972,
 3.7499629466148736]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [19]:
price_items.map(sales_tax).map(lambda x: x*.09).top(15)

[78.80226534741068,
 77.27306941706885,
 75.0724102821231,
 75.03818821907024,
 72.14530637289977,
 71.9439215154975,
 71.73155573256608,
 71.70234018072335,
 70.3259079309065,
 70.19987478473693,
 69.6895449335779,
 69.55024868656886,
 68.57305302976505,
 68.2557258862508,
 68.08677101642688]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [20]:
discounted.toDebugString()

b'(10) PythonRDD[10] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[5] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [21]:
mapped = price_items.map(lambda x: (x, x*0.92 *0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.6396508406898006, 0.529630896091155), (1.8135372791296511, 1.5016088671193513), (0.3781797936312401, 0.3131328691266668), (0.8847186600850305, 0.7325470505504053), (2.239453729133161, 1.8542676877222575), (0.2702539675560809, 0.223770285136435), (6.8764170492398256, 5.6936733167705755), (5.093166994062665, 4.217142271083888), (4.246964968318807, 3.516486993767972), (4.528940756781248, 3.7499629466148736)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [22]:
flat_mapped = price_items.flatMap(lambda x: (x, x*0.92 *0.9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.6396508406898006, 0.529630896091155, 1.8135372791296511, 1.5016088671193513, 0.3781797936312401, 0.3131328691266668, 0.8847186600850305, 0.7325470505504053, 2.239453729133161, 1.8542676877222575]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [23]:
# use the filter function
selected_items = discounted.filter(lambda x: x>300)

# calculate total remaining in inventory 
selected_items.count()

276

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [24]:
selected_items.reduce(lambda x,y: x + y)

128588.1947733079

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [25]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(20, 314.3534605097313),
 (20, 326.4994955250321),
 (33, 343.18442655367716),
 (28, 343.9850312918105),
 (37, 357.16703812307446),
 (10, 341.51590260222815),
 (3, 365.13488562948214)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [0]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x,y: x + y)

In [27]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1],ascending = False).collect()

[(20, 6899.127736381164),
 (15, 6294.376112025773),
 (24, 6133.613982404435),
 (26, 4865.793166724818),
 (50, 4497.1053941679),
 (32, 4258.795748068251),
 (40, 4024.672625943978),
 (37, 4014.365152922318),
 (36, 3643.743683855139),
 (27, 3539.0654852249854),
 (21, 3354.834140469654),
 (4, 3154.6150305157544),
 (31, 3135.522781091173),
 (16, 3123.014514805489),
 (6, 3004.2435974595573),
 (42, 2813.1075359887027),
 (22, 2773.985117740338),
 (7, 2755.2611529350634),
 (13, 2590.2492965659503),
 (48, 2589.2359335412625),
 (35, 2589.161821142071),
 (28, 2557.35186984344),
 (49, 2529.741918176156),
 (9, 2412.831750289196),
 (10, 2371.6146684821206),
 (19, 2355.002128549585),
 (38, 2343.586375828171),
 (44, 2249.4626184104545),
 (29, 2247.1036337064525),
 (8, 2140.9574029142395),
 (3, 2107.9180370906724),
 (47, 2009.2115740362633),
 (1, 1992.487874718328),
 (17, 1954.5889737412504),
 (25, 1863.8509006634995),
 (12, 1792.5254084801152),
 (5, 1751.1792489898462),
 (18, 1709.51844222418),
 (43, 1

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [0]:
# have to come back to this
num_items = sales_data.countByKey(   )

In [29]:
sorted(num_items.items(),key=lambda kv:kv[1],reverse=True)

[(40, 16),
 (2, 12),
 (48, 11),
 (35, 10),
 (24, 10),
 (11, 10),
 (1, 10),
 (47, 8),
 (21, 8),
 (50, 8),
 (39, 8),
 (7, 7),
 (4, 7),
 (46, 7),
 (17, 7),
 (15, 7),
 (29, 7),
 (19, 7),
 (13, 6),
 (43, 6),
 (6, 6),
 (45, 6),
 (28, 6),
 (9, 6),
 (8, 6),
 (16, 6),
 (44, 5),
 (26, 5),
 (42, 4),
 (37, 4),
 (34, 4),
 (49, 4),
 (14, 4),
 (10, 4),
 (36, 4),
 (30, 4),
 (41, 3),
 (27, 3),
 (3, 3),
 (32, 3),
 (12, 2),
 (22, 2),
 (31, 2),
 (25, 2),
 (5, 2),
 (33, 2),
 (20, 2)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
